패딩(padding)
  - 문장의 길이가 다를수 있으므로 병렬 연산을 위해서 모든 문장의 길이를 동일하게 맞춰야함
  - padding 이라고 하고 숫자는 0을 사용한다

In [1]:
# 텍스트 데이터를 정수로 변환(토큰화 -> 정수 인코딩)
# 가장 긴 문장의 길이를 구함(max_len)
# 짧은 문장에 0을 추가해서 max_le과 동일한 길이로 맞춤

In [ ]:
preprocess_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'],
                          ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'],
                          ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'],
                          ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
                          ['barber', 'went', 'huge', 'mountain']]